In [ ]:
df=pd.read_csv('files/UntitledFolder/gene_exp.csv')
df=df.drop(['Unnamed: 0','X1','X2','bar_name'],axis=1)
df2=pd.DataFrame()
df2=df.groupby(['x','y','UMI_Count']).agg('gene_name').apply(lambda x:" ".join(list(set(x)))).reset_index()
df3=pd.read_csv('files/UntitledFolder/link_to_img_trial.csv')
df7=df3
df7.head()

In [ ]:
df11 = pd.merge(df, df7, on=['x','y'], how='inner')
df11=df11.drop(columns=['Unnamed: 0'])
df11.head()

In [ ]:
from numpy import load
df12=pd.pivot_table(df11, values='UMI_Count', index=['Sno'],columns=['gene_name'], aggfunc=np.sum)
df13=df12.fillna(0)
intersect_col_genes= load('files/UntitledFolder/intersect_col_genes.npy',allow_pickle=True)
df13=df13[intersect_col_genes]
#df13.to_csv('Pivot_gene.csv')

## Import Data

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from numpy import array
from numpy import argmax
from keras.utils import to_categorical
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import sys
from numpy import load
from matplotlib import pyplot as plt
plt.style.use("ggplot")
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [27]:
from sklearn import preprocessing 
from numpy import log
df14=pd.read_csv('files/UntitledFolder/Pivot_gene_trial.csv')
#df14.reset_index(level=0, inplace=True)
Sno=df14['Sno']
df14.drop(['Sno'],axis=1,inplace=True)
#df14=log(df14+1)
'''min_max_scaler = preprocessing.StandardScaler()#MinMaxScaler(feature_range =(0, 1))
min_max_scaler_y = preprocessing.StandardScaler()#MinMaxScaler(feature_range =(0, 1)) 
min_max_scaler_y.fit(df14)
df14 = min_max_scaler.fit_transform(df14)'''
df14=pd.DataFrame(data=df14)
df14=df14.apply(lambda x: pd.qcut(x, 3,duplicates='drop',labels=False))
df14=df14.astype('str')
df14=df14.replace(['0', '1', '2'], ['A','B', 'C'])
df14.head()

,ABCC5,ADAM15,ADAR,ADIPOR1,AEBP1,ANXA2,ANXA6,APOC1,APRT,AQP3,...,VPS28,WBP1,WDR1,WWP1,XBP1,YIPF3,YWHAE,YWHAQ,ZFAS1,ZNF706
0,A,A,A,A,A,A,A,A,A,B,...,B,A,A,A,A,A,A,A,A,A
1,A,A,A,A,A,A,A,A,A,A,...,A,A,A,A,A,A,A,A,A,A
2,A,A,A,A,A,A,A,A,A,A,...,B,B,A,A,A,A,B,A,A,B
3,A,A,A,A,A,A,A,A,A,A,...,A,A,A,A,A,A,A,A,A,A
4,A,A,A,B,A,B,B,B,A,A,...,B,A,A,B,A,B,A,A,A,A


In [28]:
y=df14.iloc[:,:1].values
y

array([['A'],
       ['A'],
       ['A'],
       ...,
       ['C'],
       ['C'],
       ['C']], dtype=object)

In [4]:
# create a mapping of tags to integers given the loaded mapping file
def create_tag_mapping(df3):
    # create a set of all known tags
    labels = set()
    for i in range(len(df3)):
        # convert spaced separated tags into an array of tags
        gene_name = df3['gene_name'][i].split(' ')
        # add tags to the set of known labels
        labels.update(gene_name)
    # convert set of labels to a list to list
    labels = list(labels)
    # order set alphabetically
    labels.sort()
    # dict that maps labels to integers, and the reverse
    labels_map = {labels[i]:i for i in range(len(labels))}
    inv_labels_map = {i:labels[i] for i in range(len(labels))}
    return labels_map, inv_labels_map
# create a mapping of filename to tags
def create_file_mapping(df3):
    mapping = dict()
    for i in range(len(df3)):
        name, gene_name = df3['Sno'][i], df3['gene_name'][i]
        mapping[name] = gene_name.split(' ')
    return mapping
# create a one hot encoding for one list of tags
def one_hot_encode(gene_name, mapping):
    # create empty vector
    encoding = zeros(len(mapping), dtype='uint8')
    # mark 1 for each tag in the vector
    for tag in gene_name:
        encoding[mapping[tag]] = 1
    return encoding
def load_dataset(path, file_mapping, tag_mapping):
    photos, targets = list(), list()
    # enumerate files in the directory
    for filename in listdir(folder):
        # load image
        photo = load_img(path + filename, target_size=(140,140))
        # convert to numpy array
        photo = img_to_array(photo, dtype='uint8')
        # get tags
        gene_name = file_mapping[filename[:-4]]
        # one hot encode tags
        target = one_hot_encode(gene_name, tag_mapping)
        # store
        photos.append(photo)
        targets.append(target)
    X = asarray(photos, dtype='uint8')
    return X

In [5]:
from os import listdir
from numpy import zeros
from numpy import asarray
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
df5=pd.read_csv('files/UntitledFolder/Gene_list_df5.csv')
df5=df5.drop(columns=['Unnamed: 0'])
mapping_csv = df5
# create a mapping of tags to integers
tag_mapping, _ = create_tag_mapping(mapping_csv)
# create a mapping of filenames to tag ~lists
file_mapping = create_file_mapping(mapping_csv)
folder = 'files/UntitledFolder/Trainimg_breast2/'
X= load_dataset(folder, file_mapping, tag_mapping)
X=X.reshape([3501,58800]) 
print(X.shape, y.shape)

(3501, 58800) (3798, 1)


In [6]:
from sklearn.decomposition import PCA
pca = PCA(.99)
#pca = PCA(n_components=10)
principalComponents = pca.fit(X)
principalC= pca.fit_transform(X)
principalDf = pd.DataFrame(data = principalC)
#print(pca.explained_variance_ratio_)
principalDf.shape

(3501, 2350)

In [ ]:
'''correlation_matrix = principalDf.corr()
for i in range(len(correlation_matrix .columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.85:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)
            principalDf.drop(labels=correlated_features, axis=1, inplace=True)'''

### TABNET Classifier

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
Y= le.fit_transform(y)
Y=Y[0:3501]
X_train, X_test, y_train, y_test = train_test_split(principalDf, Y.ravel(), test_size = 0.15, random_state = 0, stratify =Y)
clasif = TabNetClassifier()
clasif.fit(X_train, y_train)
preds = clasif.predict(X_test)

'''min_max_scaler_x = preprocessing.StandardScaler()#MinMaxScaler(feature_range =(0, 1)) 
min_max_scaler_x.fit(principalDf)
min_max_scaler = preprocessing.StandardScaler()#MinMaxScaler(feature_range =(0, 1)) 
principalDf = min_max_scaler.fit_transform(principalDf) '''

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
#y_pred=clf.predict(X_test)
cm = confusion_matrix(y_test, preds)
print('Confusion matrix\n\n', cm)
#roc_auc = roc_auc_score(y_test, clf.predict_proba(X_test), multi_class="ovr",average='macro')
#print('Roc-auc score:',roc_auc)
print("F1-score",f1_score(y_test, preds, average='macro'))
print("Precision",precision_score(y_test, preds, average='macro'))
print("Recall",recall_score(y_test, preds, average='macro'))
print("Accuracy",accuracy_score(y_test, preds))

In [ ]:
preds

## LGBM Classifier

In [ ]:
def countFreq(arr, n):  
    mp = {}  
    for i in range(n): 
        if arr[i] not in mp: 
            mp[arr[i]] = 0
        mp[arr[i]] += 1
    for i in range(n): 
        if (mp[arr[i]] != -1): 
            print(arr[i],mp[arr[i]]) 
        mp[arr[i]] = -1  
        
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn import preprocessing 

min_max_scaler = preprocessing.MinMaxScaler(feature_range =(-1, 1)) 
X = min_max_scaler.fit_transform(X) 

le = LabelEncoder()
Y= le.fit_transform(y)



arr=Y
n = len(arr) 
countFreq(arr, n) 

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

strategy = {0:2500, 1:2500, 2:2500}
oversample = SMOTE(sampling_strategy=strategy)
X, Y = oversample.fit_resample(X, Y)
X_train, X_test, y_train, y_test = train_test_split(X[:,1:250], Y.ravel(), test_size = 0.25, random_state = 0, stratify =Y)
clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
cm = confusion_matrix(y_test, y_pred)
print('Confusion matrix\n\n', cm)
roc_auc = roc_auc_score(y_test, clf.predict_proba(X_test), multi_class="ovr",average='macro')
print('Roc-auc score:',roc_auc)
print("F1-score",f1_score(y_test, y_pred, average='macro'))
print("Precision",precision_score(y_test, y_pred, average='macro'))
print("Recall",recall_score(y_test, y_pred, average='macro'))
print("Accuracy",accuracy_score(y_test, y_pred))

## TPOT Auto ML

In [ ]:
!pip install tqdm --user
#tfds.disable_progression_bar()
#jupyter nbextension enable --py widgetsnbextension

In [ ]:
from tqdm import tqdm
import ipywidgets 
from sklearn.model_selection import RepeatedStratifiedKFold
from tpot import TPOTClassifier
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y= le.fit_transform(y)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
model = TPOTClassifier(generations=5, population_size=50, cv=cv, scoring='accuracy', verbosity=2, random_state=1, n_jobs=-1)
model.fit(principalDf, Y)

## H2o Auto ML

In [15]:
#!pip install colorama==0.4.4 --user
import h2o
from h2o.automl import H2OAutoML
h2o.init(
    nthreads=-1,     # number of threads when launching a new H2O server
    max_mem_size=12  # in gigabytes
)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "11.0.7" 2020-04-14 LTS; Java(TM) SE Runtime Environment 18.9 (build 11.0.7+8-LTS); Java HotSpot(TM) 64-Bit Server VM 18.9 (build 11.0.7+8-LTS, mixed mode)
  Starting server from /gpfs1/homes/s4634945/.local/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /state/partition1/pbs/tmpdir/pbs.476506.tinmgr2/tmpaz0s_084
  JVM stdout: /state/partition1/pbs/tmpdir/pbs.476506.tinmgr2/tmpaz0s_084/h2o_s4634945_started_from_python.out
  JVM stderr: /state/partition1/pbs/tmpdir/pbs.476506.tinmgr2/tmpaz0s_084/h2o_s4634945_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Australia/Brisbane
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,23 days
H2O_cluster_name:,H2O_from_python_s4634945_bg03n0
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,12 Gb
H2O_cluster_total_cores:,24
H2O_cluster_allowed_cores:,24
H2O_cluster_status:,"accepting new members, healthy"


In [30]:
from sklearn.preprocessing import LabelEncoder
#le = LabelEncoder()
#Y= le.fit_transform(y)
df=pd.DataFrame(data=principalDf)
df['UMI']=y[0:3501]
df=df.iloc[0:3350,:].values
df=pd.DataFrame(data=df)
train = h2o.H2OFrame(df)
x = train.columns
y1 = "2350"
x.remove(y1)
train['2350'] = train['2350'].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [31]:
aml = H2OAutoML(max_models=15, seed=1)
aml.train(x=x, y=y1, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [32]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_per_class_error,logloss,rmse,mse,auc,aucpr
DeepLearning_1_AutoML_20210224_220112,0.656024,2.45321,0.733565,0.538118,nan,nan
XGBoost_grid__1_AutoML_20210224_220112_model_2,0.66599,1.21549,0.676012,0.456992,nan,nan
DRF_1_AutoML_20210224_220112,0.66606,1.09972,0.661004,0.436927,nan,nan
GLM_1_AutoML_20210224_220112,0.666902,1.0816,0.657853,0.43277,nan,nan
StackedEnsemble_BestOfFamily_AutoML_20210224_220112,0.667832,1.08594,0.658785,0.433997,nan,nan
StackedEnsemble_AllModels_AutoML_20210224_220112,0.668103,1.083,0.657529,0.432344,nan,nan
XGBoost_grid__1_AutoML_20210224_220112_model_1,0.668253,1.24887,0.679333,0.461493,nan,nan
GBM_2_AutoML_20210224_220112,0.670166,1.1204,0.664491,0.441548,nan,nan
GBM_5_AutoML_20210224_220112,0.671874,1.11074,0.663889,0.440749,nan,nan
GBM_grid__1_AutoML_20210224_220112_model_1,0.673303,1.10383,0.662786,0.439285,nan,nan


In [33]:
train.head()

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199
-1735.8,1916.81,1190.35,136.215,396.595,479.078,-704.004,269.868,184.366,151.223,-233.273,-13.3867,742.381,709.745,1464.66,784.972,-334.628,149.465,-341.069,-174.476,-384.454,372.957,396.073,-380.51,44.9882,254.222,-214.447,-77.3286,234.886,305.71,-279.448,353.572,-467.72,-46.9827,141.623,-263.654,38.275,273.865,-344.128,-160.28,-296.171,-172.289,-155.369,232.085,322.551,-217.296,-185.128,-471.455,384.156,-135.293,161.237,41.2837,-7.91631,344.89,-203.57,-244.405,203.276,-115.635,170.647,198.815,-193.735,-122.196,-54.051,-120.722,151.631,-5.74798,-7.09617,-93.214,-386.907,150.271,103.533,-103.519,-21.7133,305.292,-147.945,-164.128,-167.893,313.454,90.5143,-4.52665,206.867,1.30048,-169.821,201.67,-103.821,69.4897,141.439,-167.178,255.104,-81.9475,93.3062,79.8971,296.094,31.0739,-81.0616,-75.2293,-18.7859,38.6992,-151.987,-47.2363,-202.427,254.216,-251.005,-38.4468,-287.75,60.7355,143.414,209.729,167.594,-140.717,-2.01877,128.742,30.3198,-93.4142,-169.997,-218.267,-171.179,-10.9385,43.239,-123.263,-18.1847,80.9657,138.473,-10.2389,-75.3772,85.4118,26.8345,1.03436,33.0181,167.049,-252.138,188.431,104.733,19.9386,141.416,121.122,-151.279,20.75,-49.5024,-56.4702,79.3763,172.008,29.7033,182.51,-104.168,-29.499,127.924,-25.8602,212.368,57.423,-218.617,-156.755,-66.1829,75.5676,127.192,-28.3858,-269.383,2.61219,40.0197,72.836,17.3805,95.0578,-100.539,-17.2523,107.952,-129.657,113.194,85.0651,427.835,-35.7436,81.6404,-63.774,-89.6454,-59.9389,84.9331,-207.958,11.7826,-26.4274,-9.41773,-92.7043,-49.2147,-217.597,-107.642,-52.7994,23.2873,-259.312,-62.7704,8.92639,-142.622,108.748,-66.4035,-116.828,-151.213,18.0332,-77.6514,16.3355,25.7061,-50.8269,-126.684,-197.909
8256.59,-228.814,1199.97,1578.12,714.135,-727.522,-263.663,132.391,1647.52,1734.65,272.855,-604.559,410.343,-657.124,1476.99,-208.817,321.625,1033.62,-216.414,-482.884,-381.12,-455.127,-137.713,1094.73,1122.78,-1135.19,-615.041,-768.324,-259.377,-636.611,-136.302,-101.277,-582.572,-417.02,250.833,-1587.63,-1304.33,1314.42,255.695,-1574.07,1701.17,675.126,-622.305,-505.459,-816.464,445.139,3.57531,747.666,-950.312,201.658,-727.585,599.493,513.49,-353.09,277.062,1042.83,-129.99,-937.511,-72.8497,-1700.01,-711.323,-817.899,311.459,420.623,-984.144,-1125.66,206.533,-1229.36,414.494,717.364,-1295.96,1255.39,496.163,819.128,-146.322,-1069.79,-1008.55,-434.918,-401.805,-430.155,-1057.61,-581.613,-316.522,-252.651,636.23,-1573.07,624.886,425.899,268.195,590.79,191.972,-570.6,1892.57,-591.286,-367.365,425.25,283.444,694.637,-653.155,386.057,75.6656,187.681,-95.6654,679.97,417.347,1188.51,-1099.6,-383.522,608.135,-225.114,1430.02,-177.185,-1722,-563.995,400.753,837.158,305.097,571.946,854.079,-436.516,135.432,600.83,607.98,-55.4355,105.436,-121.923,-337.058,-237.722,-566.81,1018.62,-350.886,311.989,338.501,-287.988,-415.626,96.1642,-11.6648,-148.117,-854.429,-316.044,646.841,444.018,64.763,-549.359,82.1267,-780.822,-122.774,128.629,230.881,-1256.01,-53.8049,-566.39,-279.276,166.681,-270.494,-62.023,-670.375,18.7271,-512.122,746.076,-252.968,450.583,-147.194,5.07358,-526.682,-660.036,32.9112,-367.926,-105.798,-74.0324,291.373,630.293,206.337,78.4135,-265.78,-341.684,434.303,353.697,313.807,-739.572,100.728,690.008,-734.61,29.9274,145.744,315.179,230.483,-311.795,-277.973,575.244,-533.556,-3.86468,-80.8102,445.6

In [34]:
test=pd.DataFrame(data=principalDf)
test['UMI']=y[0:3501]
test=test.iloc[3350:,:-1].values
test=pd.DataFrame(data=test)
test = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [35]:
import seaborn as sns
preds = aml.leader.predict(test)
pred = h2o.as_list(preds, use_pandas=True)
#sns.lineplot(data=y[3350:])
#sns.lineplot(data=pred)
pred

deeplearning prediction progress: |███████████████████████████████████████| 100%


,predict,A,B,C
0,A,9.989075e-01,0.000005,0.001087
1,A,6.822846e-01,0.241947,0.075768
2,C,7.550037e-02,0.000255,0.924245
3,A,9.731309e-01,0.026512,0.000357
4,A,9.177736e-01,0.072987,0.009239
...,...,...,...,...
146,C,4.027726e-03,0.005101,0.990871
147,C,1.788934e-09,0.000016,0.999984
148,C,2.140450e-01,0.018453,0.767502
149,A,5.845253e-01,0.061359,0.354115


In [42]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
cm = confusion_matrix(y[3350:3501], pred['predict'])
print('Confusion matrix\n\n', cm)
print("F1-score",f1_score(y[3350:3501,], pred['predict'], average='macro'))
print("Precision",precision_score(y[3350:3501], pred['predict'], average='macro'))
print("Recall",recall_score(y[3350:3501], pred['predict'], average='macro'))
print("Accuracy",accuracy_score(y[3350:3501], pred['predict']))

Confusion matrix

 [[ 0  1  1]
 [18  9 11]
 [50 22 39]]
F1-score 0.2462081128747796
Precision 0.3486519607843137
Recall 0.19606448553816977
Accuracy 0.31788079470198677


## LGBM Regressor

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn import preprocessing
#from sklearn.preprocessing import StandardScaler, PolynomialFeatures, Imputer
from sklearn.model_selection import cross_val_score, cross_val_predict


hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['l2', 'auc'],
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    "max_depth": 32,
    "num_leaves": 256,  
    "max_bin": 1024,
    "num_iterations": 100000,
    "n_estimators": 1000
}

In [ ]:
trainX, testX, trainY, testY = train_test_split(principalDf, y, test_size=0.20, random_state=1)

trainY=trainY.reshape([2725,])
trainY=trainY.ravel()
testY=testY.reshape([682,])
testY=testY.ravel()

gbm = lgb.LGBMRegressor(**hyper_params)
gbm.fit(trainX, trainY,
        eval_set=[(testX, testY)],
        eval_metric='l2',
        early_stopping_rounds=1000)
y_pred = gbm.predict(testX, num_iteration=gbm.best_iteration_)

In [ ]:
plt.scatter(x=trainX[2], y=trainY, color ='blue')
plt.scatter(x=testX[2], y=y_pred, color ='red')

import sklearn.metrics as sm
print("Mean absolute error =", round(sm.mean_absolute_error(testY, y_pred), 2)) 
print("Mean squared error =", round(sm.mean_squared_error(testY, y_pred), 2)) 
print("Median absolute error =", round(sm.median_absolute_error(testY, y_pred), 2)) 
print("Explain variance score =", round(sm.explained_variance_score(testY, y_pred), 2)) 
print("R2 score =", round(sm.r2_score(testY, y_pred), 2))

## Multivariate LGBM Regressor

In [ ]:
from numpy import mean
from numpy import std
from numpy import absolute
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import LinearSVR
import lightgbm as lgb

hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['l2', 'auc'],
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    "max_depth": 8,
    "num_leaves": 128,  
    "max_bin": 512,
    "num_iterations": 100000,
    "n_estimators": 1000
}

y=df13.iloc[:,0:5].values
y=y.astype(int)

model = lgb.LGBMRegressor(**hyper_params)
wrapper = MultiOutputRegressor(model)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(wrapper, principalDf, y, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)
n_scores = absolute(n_scores)
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [ ]:
wrapper = MultiOutputRegressor(model)
wrapper.fit(X, y)
yhat = wrapper.predict(testY)

In [ ]:
import sklearn.metrics as sm
print("Mean absolute error =", round(sm.mean_absolute_error(testY, y_pred), 2)) 
print("Mean squared error =", round(sm.mean_squared_error(testY, y_pred), 2)) 
print("Median absolute error =", round(sm.median_absolute_error(testY, y_pred), 2)) 
print("Explain variance score =", round(sm.explained_variance_score(testY, y_pred), 2)) 
print("R2 score =", round(sm.r2_score(testY, y_pred), 2))

### Polynomial Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR

#X = df.iloc[:, :-1].values
#y = df.iloc[:, 58800].values
#y=df13.iloc[:,:1].values
#y=y.astype(int)

trainX, testX, trainY, testY = train_test_split(principalDf, y, test_size=0.20, random_state=1)
sc = StandardScaler()
trainX = sc.fit_transform(trainX)
testX = sc.transform(testX)
#trainY = sc.fit_transform(trainY)
#testY = sc.fit_transform(testY)
poly_reg = PolynomialFeatures(degree = 5)
X_poly = poly_reg.fit_transform(trainX)
poly_reg.fit(X_poly, trainY)

y_pred = regressor.predict(testX)

In [ ]:
import seaborn as sns
#preds = model.predict(testX)
'''y_preds = y_pred
plt.scatter(x=list(range(0, 682)), y=testY, color ='blue')
plt.scatter(x=list(range(0, 682)), y=y_preds, color ='red')'''
sns.lineplot(data=testY)
sns.lineplot(data=y_pred)

In [ ]:
import sklearn.metrics as sm
print("Mean absolute error =", round(sm.mean_absolute_error(testY, y_pred), 2)) 
print("Mean squared error =", round(sm.mean_squared_error(testY, y_pred), 2)) 
print("Median absolute error =", round(sm.median_absolute_error(testY, y_pred), 2)) 
print("Explain variance score =", round(sm.explained_variance_score(testY, y_pred), 2)) 
print("R2 score =", round(sm.r2_score(testY, y_pred), 2))

# AutoML

In [ ]:
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from autosklearn.regression import AutoSklearnRegressor
from autosklearn.metrics import mean_absolute_error as auto_mean_absolute_error
from sklearn.model_selection import train_test_split

In [ ]:
trainX, testX, trainY, testY = train_test_split(principalDf, y, test_size=0.20, random_state=1)
# define search
model = AutoSklearnRegressor(time_left_for_this_task=5*60, per_run_time_limit=30, n_jobs=8)
# perform the search
model.fit(trainX, trainY)

## TBANET-Regressor

In [ ]:
import pandas as pd
import numpy  as np
import torch
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import KFold

In [ ]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(principalDf, y, test_size=0.20, random_state=1)
X      = trainX.to_numpy()
#y      = trainY.to_numpy()
X_test = testX.to_numpy()
kf = KFold(n_splits=2, random_state=42, shuffle=True)
predictions_array =[]
CV_score_array    =[]
for train_index, test_index in kf.split(X):
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    regressor = TabNetRegressor(n_d=512,n_a=512,scheduler_fn=torch.optim.lr_scheduler.MultiStepLR,scheduler_params = {"milestones": [150,250,300,350,400,450],'gamma':0.2},verbose=1,seed=42)
    regressor.fit(X_train=X_train, y_train=y_train,
              eval_set=[(X_valid, y_valid)],
              patience=200, max_epochs=20,
              eval_metric=['rmse'])
    CV_score_array.append(regressor.best_cost)
    predictions_array.append(regressor.predict(X_test))
predictions = np.mean(predictions_array,axis=0)

In [ ]:
import sklearn.metrics as sm
print("Mean absolute error =", round(sm.mean_absolute_error(testY, predictions_array[1]), 2)) 
print("Mean squared error =", round(sm.mean_squared_error(testY, predictions_array[1]), 2)) 
print("Median absolute error =", round(sm.median_absolute_error(testY, predictions_array[1]), 2)) 
print("Explain variance score =", round(sm.explained_variance_score(testY, predictions_array[1]), 2)) 
print("R2 score =", round(sm.r2_score(testY, predictions_array[1]), 2))

## H20 AutoML

In [ ]:
#!pip install colorama==0.4.4 --user
import h2o
from h2o.automl import H2OAutoML

In [ ]:
h2o.init(
    nthreads=-1,     # number of threads when launching a new H2O server
    max_mem_size=12  # in gigabytes
)

In [ ]:
df=pd.DataFrame(data=principalDf)
df['UMI']=y
df=df.iloc[0:3350,:].values
df=pd.DataFrame(data=df)
train = h2o.H2OFrame(df)
x = train.columns
y1 = "2228"
x.remove(y1)

In [ ]:
aml = H2OAutoML(max_models=15, seed=1)
aml.train(x=x, y=y1, training_frame=train)

In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

In [ ]:
test=pd.DataFrame(data=principalDf)
test['UMI']=y
test=test.iloc[3350:,:-1].values
test=pd.DataFrame(data=test)
test = h2o.H2OFrame(test)
test.head()

In [ ]:
#preds = aml.leader.predict(test)
import seaborn as sns
pred = h2o.as_list(preds, use_pandas=True)
sns.lineplot(data=y[3350:])
sns.lineplot(data=pred)